# Imports: 

In [1]:
import pandas as pd
import rdflib
import os

from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import FOAF, XSD

This script is to take the MTb proteome and fetch the RheaID for each gene. We then use the rhea_Test to fetch the ChebiID and the smiles string from the Rhea ID for each. 

# Functions: 

This function looks at the 'g', the graph output from running the rhea database to pair the substrates and products from the rhea ID.

In [2]:
def get_chebi_side_rxn(rxn_id_side): 
    
    list_part_comps = []
    for obj in g.objects(rxn_id_side, contains_pred):
        list_part_comps.append(obj)

    list_comps = []
    for part_comp in list_part_comps:
        for obj in g.objects(part_comp, compound_pred):
            list_comps.append(obj)

#     list_chebi = []
#     for comp in list_comps:
#         for obj in g.objects(comp, accession_pred):
#             list_chebi.append(obj)

    list_chebi = []
    for comp in list_comps:
        for obj in g.objects(comp, accession_pred):

            if (str(obj).split(':')[0] == 'GENERIC'):
                print('WARNING: generic compound')

                list_reactive = []
                for obj in g.objects(comp, reactive_part_pred):
                    list_reactive.append(obj)

                for reactive in list_reactive:
                    for obj in g.objects(reactive, chebi_pred):
                          list_chebi.append(str(obj).split('/')[-1])

            else:
                list_chebi.append( '_'.join(str(obj).split(':')) )
    
    return list_chebi

This function adds the rh link to each RHEA ID and adds either "L" or "R" to end of the string, to look up either the substrates (L or left) or the products (R or right) of the reaction.

In [3]:
def get_chebi_from_rhea_ID(RHEA_id):

    rxn_id = URIRef(rh+RHEA_id)
    rxn_id_L = URIRef(rh+RHEA_id+'_L')
    rxn_id_R = URIRef(rh+RHEA_id+'_R')

    list_chebi_L = get_chebi_side_rxn(rxn_id_L)
    list_chebi_R = get_chebi_side_rxn(rxn_id_R)
            
    return list_chebi_L, list_chebi_R

In [31]:
R_id = str(21528)
get_chebi_from_rhea_ID(R_id)

(['POLYMER_9859', 'CHEBI_15377'],
 ['POLYMER_14279', 'CHEBI_15378', 'CHEBI_43474'])

In [36]:
rxn_id = URIRef(rh+R_id)
rxn_id_L = URIRef(rh+R_id+'_L')
rxn_id_R = URIRef(rh+R_id+'_R')
get_chebi_side_rxn(rxn_id_L)

list_part_comps = []
for obj in g.objects(rxn_id_L, contains_pred):
    list_part_comps.append(obj)
print(list_part_comps)

list_comps = []
for part_comp in list_part_comps:
    for obj in g.objects(part_comp, compound_pred):
        list_comps.append(obj)
        
print(list_comps)

[rdflib.term.URIRef('http://rdf.rhea-db.org/Participant_21528_compound_9859'), rdflib.term.URIRef('http://rdf.rhea-db.org/Participant_21528_compound_1283')]
[rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_9859'), rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_1283')]


# Parameters: 

In [4]:
rh = 'http://rdf.rhea-db.org/'
# predicates: 
substrates_pred = URIRef(rh+'substrates')
contains_pred = URIRef(rh+'contains')
contains1_pred = URIRef(rh+'contains1')
accession_pred = URIRef(rh+'accession')
compound_pred = URIRef(rh+'compound')
side_pred = URIRef(rh+'side')
directional_pred = URIRef(rh+'directionalReaction')

reactive_part_pred = URIRef(rh+'reactivePart')
chebi_pred = URIRef(rh+'chebi')

In [5]:
# CHEBI:16526 = CO2
# 'CHEBI_57945' = , 'CHEBI_15378' = , 'CHEBI_57540' = , 'CHEBI_58349' = , 'CHEBI_57783 = '
cofactor_IDS = ['CHEBI_57945', 'CHEBI_15378', 'CHEBI_57540', 'CHEBI_58349', 'CHEBI_57783', 'CHEBI_16526']

# Read in files: 

#### Rhea database: 
We are uploading the entire Rhea Database and storing it in "g". This allows us to parse through all the entries and connect the RheaID in our df_UP to CHEBI IDs and smile strings. 

In [8]:
#Upload the Rhea Database.
path = '/home/sakila/Documents/Data/rhea_test'
file_in = os.path.join(path, 'rhea.rdf') 

# Rdflib reads through the rdf file and creates a graph connecting the substrates to the proteins and RheaIDs. 
g = rdflib.Graph()
result = g.parse(file_in)

print(len(g))

1327346


#### ChEBI-to-SMILES database: 

In [9]:
file_chebi = os.path.join(path, 'chebi.owl')
# create a Graph
g_chebi = rdflib.Graph()
result_chebi = g_chebi.parse(file_chebi)

len(g_chebi)

2340334

#### UniProt-to-Rhea dataset:
This was generated in the notebook "EDA_rhea-Sakila.ipynb"

This dataframe has the UniProt ID's for all proteins with:
1. all proteins with a known substrate

In [10]:
file_in = '../data/up_to_rhea.csv'
df_UP_rhea = pd.read_csv(file_in)
df_UP_rhea.head(2)

,Entry,RheaID
0,P9WJM5,14217
1,P9WJM5,19993


# Part I: Mapping Rhea-to-ChEBI: 

##### Get unique Rhea reaction IDs:

In [12]:
list_rhea_IDs = df_UP_rhea.RheaID.unique().tolist()
len(list_rhea_IDs)

1217

##### Map Rhea reaction ID's to ChEBI substrate/product ID's: 

[QUESTION]: What does the "generic compound" WARNING mean? 

In [ ]:
list_chebi_LR_ALL = []
list_rhea_IDs_ALL = []
list_type_ALL = []

for RHEA_id in list_rhea_IDs: # for each Rhea reaction ID: 
    list_chebi_L, list_chebi_R = get_chebi_from_rhea_ID(str(RHEA_id)) # Get left and right compounds
#     list_chebi_L = ['_'.join(str(chebi).split(':')) for chebi in list_chebi_L if str(chebi) not in cofactor_IDS]
#     list_chebi_R = ['_'.join(str(chebi).split(':')) for chebi in list_chebi_R if str(chebi) not in cofactor_IDS]
    # In this version, we don't discard the cofactors --> this might be important information to "learn" from the protein sequences. 
    list_chebi_L = ['_'.join(str(chebi).split(':')) for chebi in list_chebi_L ]
    list_chebi_R = ['_'.join(str(chebi).split(':')) for chebi in list_chebi_R ]
    
    list_chebi_LR_ALL += list_chebi_L
    list_chebi_LR_ALL += list_chebi_R
    
    list_rhea_IDs_ALL += [RHEA_id]*len(list_chebi_L)
    list_rhea_IDs_ALL += [RHEA_id]*len(list_chebi_R)
    
    list_type_ALL += ['LEFT']*len(list_chebi_L)
    list_type_ALL += ['RIGHT']*len(list_chebi_R)


Store in dataframe: 

In [16]:
df_Rhea_ChEBI = pd.DataFrame()
df_Rhea_ChEBI['RheaID'] = list_rhea_IDs_ALL
df_Rhea_ChEBI['type'] = list_type_ALL
df_Rhea_ChEBI['ChEBI'] = list_chebi_LR_ALL

In [39]:
df_Rhea_ChEBI.head()


,RheaID,type,ChEBI
0,14217,LEFT,CHEBI_50058
1,14217,LEFT,CHEBI_15377
2,14217,LEFT,CHEBI_16044
3,14217,RIGHT,CHEBI_29950
4,14217,RIGHT,CHEBI_29950


In [51]:
df_Rhea_ChEBI[df_Rhea_ChEBI.ChEBI.str.contains("POLYMER")].ChEBI.unique().shape

(31,)

In [47]:
df_chebi_smiles[df_chebi_smiles.ChEBI.str.contains("POLYMER")]

,ChEBI,smiles
28,POLYMER_9859,NaN
29,POLYMER_14279,NaN
88,POLYMER_11133,NaN
89,POLYMER_11132,NaN
90,POLYMER_11129,NaN
91,POLYMER_11128,NaN
284,POLYMER_14280,NaN
290,POLYMER_9527,NaN
292,POLYMER_9517,NaN
410,POLYMER_9537,NaN


In [37]:
fn = "../data/tests/df_Rhea_CheBI.xlsx"
df_Rhea_ChEBI.to_excel(fn, index = False)

#### Merging the two dataframes? 
1. The UniProt to RheaID mappings ("df_UP_rhea"), with
2. The RheaID to ChEBI compound ID mappings: 

In [19]:
df_UP_rhea

,Entry,RheaID
0,P9WJM5,14217
1,P9WJM5,19993
2,P9WHM9,17453
3,P9WGE9,20696
4,P9WG79,24840


In [21]:
df_UP_rhea_chebi = df_UP_rhea.merge(df_Rhea_ChEBI, on = 'RheaID', how = 'inner')
#Find the common columns with the "on". When you do inner, you only keep the intersection of the 2 data frames, while "left' keeps the rows and adds NaN.
print(df_UP_rhea_chebi.shape)
df_UP_rhea_chebi.head(5)

(7365, 4)


,Entry,RheaID,type,ChEBI
0,P9WJM5,14217,LEFT,CHEBI_50058
1,P9WJM5,14217,LEFT,CHEBI_15377
2,P9WJM5,14217,LEFT,CHEBI_16044
3,P9WJM5,14217,RIGHT,CHEBI_29950
4,P9WJM5,14217,RIGHT,CHEBI_29950


#### Mini-excercise: 
These are the Rhea reactions in the database, ranked by how often they appear? 
What can we learn from this? 

In [18]:
df_UP_rhea_chebi.RheaID.value_counts().head(15)

28286    1250
42640    1230
48096     115
12721      80
17877      80
24604      60
54724      55
54712      55
54736      55
42708      52
20225      50
15573      50
50904      45
43152      45
36899      45
Name: RheaID, dtype: int64

### ChEBI compound ID's to SMILES: 

In [23]:
uri = 'http://purl.obolibrary.org/obo/'

# rdf relevant predicates: 
smiles_pred = URIRef(uri+'chebi/smiles')

Testing to get a smiles string for a single compound

In [24]:
#CReating a uri for each Chebi_ID and then a generic uri that you need to tell the program that you're interested in the smiles string.
#This searches the 'g' database for the smiles strings associated with the 
chebi_id = 'CHEBI_58349'
chebi_id_uri = URIRef(uri+chebi_id)
list_smiles = []
for obj in g_chebi.objects(chebi_id_uri, smiles_pred):
    list_smiles.append(str(obj)) 

In [26]:
chebi_id_uri
smiles_pred

rdflib.term.URIRef('http://purl.obolibrary.org/obo/chebi/smiles')

In [444]:
list_smiles

['NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OC[C@H]2O[C@H]([C@H](OP([O-])([O-])=O)[C@@H]2O)n2cnc3c(N)ncnc23)[C@@H](O)[C@H]1O']

In [27]:
list_chebi = df_UP_rhea_chebi.ChEBI.unique().tolist()
len(list_chebi)

1174

In [29]:
df_UP_rhea_chebi[df_UP_rhea_chebi.ChEBI == 'POLYMER_9859']

,Entry,RheaID,type,ChEBI
41,P9WHV5,21528,LEFT,POLYMER_9859
46,P96374,21528,LEFT,POLYMER_9859
1514,O05877,55412,LEFT,POLYMER_9859
1994,P9WIN1,22036,LEFT,POLYMER_9859
2696,P9WHV9,19573,LEFT,POLYMER_9859


In [28]:
list_smiles_ALL = []
for chebi_id in list_chebi:
    
    chebi_id_uri = URIRef(uri+chebi_id)
    #SOme CHebIDs return more than one smiles, so you create a list of smiles for each chebi ID.
    list_smiles = []
    for obj in g_chebi.objects(chebi_id_uri, smiles_pred):
        list_smiles.append(str(obj)) 
        
    #If there are no smiles associated with it, then you would print 'no smiles'.
    if len(list_smiles)==0:
        print(chebi_id, 'zero smiles')
        list_smiles.append('NaN')
    if len(list_smiles)>1:
        print(chebi_id, 'more than 1 smiles')
        
    list_smiles_ALL+=list_smiles
        

POLYMER_9859 zero smiles
POLYMER_14279 zero smiles
POLYMER_11133 zero smiles
POLYMER_11132 zero smiles
POLYMER_11129 zero smiles
POLYMER_11128 zero smiles
POLYMER_14280 zero smiles
POLYMER_9527 zero smiles
POLYMER_9517 zero smiles
CHEBI_84139 zero smiles
POLYMER_9537 zero smiles
POLYMER_9539 zero smiles
POLYMER_9565 zero smiles
POLYMER_9566 zero smiles
POLYMER_9602 zero smiles
POLYMER_9603 zero smiles
POLYMER_9584 zero smiles
POLYMER_9586 zero smiles
POLYMER_11130 zero smiles
POLYMER_11131 zero smiles
POLYMER_12939 zero smiles
POLYMER_14378 zero smiles
POLYMER_12940 zero smiles
POLYMER_13431 zero smiles
POLYMER_9587 zero smiles
POLYMER_14738 zero smiles
POLYMER_14740 zero smiles
POLYMER_9564 zero smiles
POLYMER_9563 zero smiles
POLYMER_10183 zero smiles
POLYMER_14708 zero smiles
POLYMER_14709 zero smiles


In [44]:
list_smiles_ALL[:5]

['C([C@@H](N*)CSSC[C@@H](C(=O)*)N*)(=O)*',
 '[H]O[H]',
 'O=C(*)[C@@H](N*)CCSC',
 'C(=O)(*)[C@@H](N*)CS',
 'C(*)(=O)[C@@H](N*)CC[S@@](=O)C']

In [45]:
df_chebi_smiles = pd.DataFrame()
df_chebi_smiles['ChEBI'] = list_chebi
df_chebi_smiles['smiles'] = list_smiles_ALL
df_chebi_smiles.head(2)

,ChEBI,smiles
0,CHEBI_50058,C([C@@H](N*)CSSC[C@@H](C(=O)*)N*)(=O)*
1,CHEBI_15377,[H]O[H]


In [46]:
df_chebi_smiles.shape

(1174, 2)

#### One final merge: 

In [33]:
df_UP_rhea_chebi_smiles = df_UP_rhea_chebi.merge(df_chebi_smiles, on = 'ChEBI', how = 'inner')

In [34]:
df_UP_rhea_chebi_smiles.sort_values(by = ['Entry', 'RheaID', 'type'], inplace=True)
df_UP_rhea_chebi_smiles.reset_index(drop=True, inplace=True)

In [35]:
df_UP_rhea_chebi_smiles.shape

(6036, 5)

In [37]:
df_UP_rhea_chebi_smiles.head(3)

,Entry,RheaID,type,ChEBI,smiles
0,A0A077EWA5,51268,LEFT,CHEBI_59789,C[S+](CC[C@H]([NH3+])C([O-])=O)C[C@H]1O[C@H]([...
1,A0A077EWA5,51268,LEFT,CHEBI_134001,C1(=C(C=C(C=C1)C[NH2+]CCC2=CC=C(C=C2)O)O)O
2,A0A077EWA5,51268,RIGHT,CHEBI_15378,[H+]


In [38]:
file_out = '../../data/mohammed/SAM_UP45_Rhea_ChEBI_smiles.csv'
df_UP_rhea_chebi_smiles.to_csv(file_out, index = False)

### Same database above, but discarding NAD(H), NADP(H), CO2, H+, (others?)

In [ ]:
cofactor_IDS = ['CHEBI_57945', 'CHEBI_15378', 'CHEBI_57540', 'CHEBI_58349', 'CHEBI_57783', 'CHEBI_16526']

In [473]:
df_UP_rhea_chebi_smiles_noCF = df_UP_rhea_chebi_smiles[~df_UP_rhea_chebi_smiles.ChEBI.isin(cofactor_IDS)].copy()
df_UP_rhea_chebi_smiles_noCF.reset_index(inplace=True, drop=True)

In [482]:
len(df_UP_rhea_chebi_smiles_noCF['ChEBI'].unique().tolist())

425

In [480]:
len(df_UP_rhea_chebi_smiles_noCF['smiles'].unique().tolist())

421

In [475]:
df_UP_rhea_chebi_smiles_noCF.head(10)

,Entry,RheaID,type,ChEBI,smiles
0,A0A024R7X6,25033,LEFT,CHEBI_17336,CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/CO
1,A0A024R7X6,25033,RIGHT,CHEBI_17898,[H]C(=O)\C=C(C)\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C
2,A0A075TRB3,62216,LEFT,CHEBI_145109,C1(C(=C[C@@H]([C@@H]2[C@H]1O2)O)CO)=O
3,A0A075TRB3,62216,RIGHT,CHEBI_145110,C1(C(=CC([C@@H]2[C@H]1O2)=O)CO)=O
4,A0A096NEU8,25033,LEFT,CHEBI_17336,CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/CO
5,A0A096NEU8,25033,RIGHT,CHEBI_17898,[H]C(=O)\C=C(C)\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C
6,A0A0D9RMB0,25033,LEFT,CHEBI_17336,CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/CO
7,A0A0D9RMB0,25033,RIGHT,CHEBI_17898,[H]C(=O)\C=C(C)\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C
8,A0A0L8VV04,48680,LEFT,CHEBI_85440,CC(C)(COP([O-])(=O)OP([O-])(=O)OC[C@H]1O[C@H](...
9,A0A0L8VV04,48680,RIGHT,CHEBI_90725,[C@@H]1(N2C3=C(C(=NC=N3)N)N=C2)O[C@H](COP(OP(O...


## Exercise 1: NAD vs. NADP:

## Exercise 2: Reactions involving CO2

# Dig a bit into this: 

You might need to discard some of these (decide on a case-by-case basis):

In [310]:
df_Rhea_ChEBI.RheaID.value_counts().head(7)

19397    6
15805    5
38575    4
12044    4
12456    4
18833    3
14981    2
Name: RheaID, dtype: int64

In [311]:
df_UP_rhea[df_UP_rhea.RheaID.values == '12044']

,Entry,RheaID
241,Q09851,12044
1098,P40471,12044


And what exactly is the meaning and what are the consequences of this? 

In [315]:
df_Rhea_ChEBI[df_Rhea_ChEBI.ChEBI.str.contains('GENERIC')].head(5)

,RheaID,type,ChEBI
10,48488,LEFT,GENERIC_14428
11,48488,RIGHT,GENERIC_14430
187,17397,LEFT,GENERIC_9945
188,17397,RIGHT,GENERIC_9916
316,22564,LEFT,GENERIC_9926


In [325]:
RHEA_id = '48488'

In [326]:
df_Rhea_ChEBI[df_Rhea_ChEBI.RheaID.values == RHEA_id]

,RheaID,type,ChEBI
10,48488,LEFT,GENERIC_14428
11,48488,RIGHT,GENERIC_14430


In [320]:
df_UP_rhea[df_UP_rhea.RheaID.values == rid].head(2)

,Entry,RheaID
7,O54753,48488
37,O54909,48488


In [321]:
rxn_id = URIRef(rh+RHEA_id)
rxn_id_L = URIRef(rh+RHEA_id+'_L')
rxn_id_R = URIRef(rh+RHEA_id+'_R')

In [368]:
reactive_part_pred = URIRef(rh+'reactivePart')
chebi_pred = URIRef(rh+'chebi')

In [385]:
rxn_id_side = rxn_id_R

In [386]:
list_part_comps = []
for obj in g.objects(rxn_id_side, contains_pred):
    list_part_comps.append(obj)

list_comps = []
for part_comp in list_part_comps:
    for obj in g.objects(part_comp, compound_pred):
        list_comps.append(obj)
        
list_chebi = []
for comp in list_comps:
    for obj in g.objects(comp, accession_pred):
        
        if (str(obj).split(':')[0] == 'GENERIC'):
            print('WARNING: generic compound')
            
            list_reactive = []
            for obj in g.objects(comp, reactive_part_pred):
                list_reactive.append(obj)
            
            for reactive in list_reactive:
                for obj in g.objects(reactive, chebi_pred):
                      list_chebi.append(str(obj).split('/')[-1])
                 
        else:
            list_chebi.append( '_'.join(str(obj).split(':')) )

In [387]:
list_part_comps

[rdflib.term.URIRef('http://rdf.rhea-db.org/Participant_48488_compound_3249'),
 rdflib.term.URIRef('http://rdf.rhea-db.org/Participant_48488_compound_1650'),
 rdflib.term.URIRef('http://rdf.rhea-db.org/Participant_48488_compound_14430')]

In [388]:
list_comps

[rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_3249'),
 rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_1650'),
 rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_14430')]

In [389]:
list_chebi

['CHEBI_15378', 'CHEBI_57945', 'CHEBI_83228', 'CHEBI_17898']

In [357]:
for pred_obj in g.predicate_objects(list_comps[1]):
    print(pred_obj)

(rdflib.term.URIRef('http://rdf.rhea-db.org/id'), rdflib.term.Literal('14428', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://rdf.rhea-db.org/reactivePart'), rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_14428_rp2'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/htmlName'), rdflib.term.Literal('<i>all</i>-<i>trans</i>-retinol--[retinol-binding protein]'))
(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://rdf.rhea-db.org/GenericPolypeptide'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/accession'), rdflib.term.Literal('GENERIC:14428'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/reactivePart'), rdflib.term.URIRef('http://rdf.rhea-db.org/Compound_14428_rp1'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/name'), rdflib.term.Literal('all-trans-retinol--[retinol-binding protein]'))


This is what you eventually need to fetch (if you run into a compound that's only returning "generic")

In [349]:
for obj in g.objects(comp, reactive_part_pred):
    print(obj)

http://rdf.rhea-db.org/Compound_14428_rp2
http://rdf.rhea-db.org/Compound_14428_rp1


In [363]:
cid = URIRef('http://rdf.rhea-db.org/Compound_14428_rp1')
for pred_obj in g.predicate_objects(cid):
    print(pred_obj)

(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://rdf.rhea-db.org/ReactivePart'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/name'), rdflib.term.Literal('all-trans-retinol'))
(rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_17336'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/formula'), rdflib.term.Literal('C20H30O'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/htmlName'), rdflib.term.Literal('<i>all</i>-<i>trans</i>-retinol'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/charge'), rdflib.term.Literal('0'))
(rdflib.term.URIRef('http://rdf.rhea-db.org/chebi'), rdflib.term.URIRef('http://purl.obolibrary.org/obo/CHEBI_17336'))


# Misc: 

In [ ]:
for pred_obj in g_chebi.predicate_objects(chebi_id_uri):
    print(pred_obj)